# Welcome to "table_generate.ipynb"

### Here is a basic user guide:

This notebook creates tables based on a csv. To do this, a few basic inputs are needed. First, place the csv you wish to convert into the "data" folder. This csv should have the following format: row one: column names, row two: column units, row three: column descriptions. Once this csv is in place, open config.ini (located within the same folder as this file. it can be opened with notepad or a similar program). Change the properties so they match the following pattern:

```
[paper]
title = **title of paper**
authors = **authors of paper**

[**file name**]
alternate_output_name = *if you want the output name to be different from the file name, put it here, else put False*
table_name = **name of table**
RA_column = **index (starting at 0) of the column in the table that has the RA in decimal degrees**
DEC_column = **index (starting at 0) of the column in the table that has the DEC in decimal degrees**
```

The file properties can be repeated as many times as necessary, however they should be in the order they appear in the paper. Here is an example of a config:

```
[paper]
title = ACCEPT 2.0: A compilation of X-ray gas properties for clusters of galaxies observed by the Chandra X-ray Observatory
authors = Donahue M., Baldi A., Frisbie R., Koeppe D., Voit G.M.

[globalTestFormat.csv]
alternate_output_name = global_properties_test
table_name = Global properties
RA_column = 8
DEC_column = 9

[deprojected_profiles_arcsec_Alessandro_reformated.csv]
alternate_output_name = deprojected_profiles_test
table_name = Deprojected Profiles
RA_column = 21
DEC_column = 22
```

Once these are properly set, simply run both cells of code and the output will be stored in a folder in the "table_generate_output" folder

In [108]:
from astropy.table import Table
import pandas as pd
import numpy as np
import tempfile
import os
import shutil
import re
from datetime import date

class table_generate:
    def __init__(self,index,paper,table,input_file,batch_number): #intializes the table with all the properties from the config file
        self.index = index

        self.title = paper['title']
        self.authors = paper['authors']

        self.alternate_output_name = table['alternate_output_name']
        self.table_name = table['table_name']
        self.RA_column = table['ra_column']
        self.DEC_column = table['dec_column']

        self.file_loc = r'../data/'+input_file
        self.input_file=input_file

        self.raw_name = input_file[:-4]
        if self.alternate_output_name:
            self.raw_name = self.alternate_output_name

        os.makedirs(r'table_generate_output/'+str(date.today())+'_batch'+str(batch_number)+r'/'+self.raw_name) #creates the output folder
        self.output_path = r'table_generate_output/'+str(date.today())+'_batch'+str(batch_number)+r'/'+self.raw_name+r'/'
    
    def make_mrt(self):
        df = pd.read_csv(self.file_loc)
        unitmap = df.to_dict(orient='records')[0] #create a map for the units of each column
        descriptionmap = df.to_dict(orient='records')[1] #create a map for the description of each column
        df = df.drop([0]) #drops the units row
        df = df.drop([1]) #drops the description row

        tempcsv = tempfile.NamedTemporaryFile() #writes a temporary csv file to be read by astropy, this fixes an issue where directly feeding astropy a pandas df doesn't correctly assign datatypes
        df.to_csv(tempcsv,index=False)
        csv = Table.read(tempcsv,format='csv',units=unitmap,descriptions=descriptionmap) #read the csv into an astropy table

        csv.write(self.output_path+'tab'+str(self.index)+'.txt', overwrite=True, format='mrt') #uses astropy to write and mrt lacking metadata

        mrt_replaceables = { #used to replace empty metadata in the mrt
            'Title:': 'Title: ' + self.title,
            'Authors:': 'Authors: '+ self.authors,
            'Table:': 'Table: '+ self.table_name,
            'table.dat': self.raw_name+'.dat',
        }

        with open(self.output_path+'tab'+str(self.index)+'.txt', 'r+') as file: #replaces empty MRT metadata
            read_content=file.read()
            #read_content=re.sub("[\[].*?[\]] ", "", read_content) #gets rid of limits, wish this was just an option when exporting
            for key in mrt_replaceables:
                read_content=read_content.replace(key, mrt_replaceables[key])
            file.seek(0)
            file.write(read_content)
        
    def make_html(self):
        pd.read_csv(self.file_loc,header=[0,1],skiprows=[2]).to_html(self.output_path+self.raw_name+'.html',index=False) #makes the html table via pandas
    
    def make_conesearch(self):
        shutil.copy('search.php', self.output_path)
        shutil.copy(self.file_loc, self.output_path)

        consearch_replaceables = { #used to fill in info for conesearch script
            'file_name_replace': '"' + self.input_file + '"',
            'RA_index_replace': self.RA_column,
            'DEC_index_replace': self.DEC_column
        }

        with open(self.output_path+'search.php', 'r+') as file: #fills in necessary info for conesearch script
            read_content=file.read()
            for key in consearch_replaceables:
                read_content=read_content.replace(key, consearch_replaceables[key])
            file.seek(0)
            file.write(read_content)

    def make_all(self):
        self.make_mrt()
        self.make_html()
        self.make_conesearch()

In [110]:
import configparser
config = configparser.ConfigParser() 
config.read('config.ini') #gets the configuration from the ini file

paper_config = dict(config.items('paper')) #grabs the paper configs

batch_number=1
while os.path.exists(r'table_generate_output/'+str(date.today())+'_batch'+str(batch_number)): #tess for a folder based on date and number of batches existing for today
    batch_number+=1
os.makedirs(r'table_generate_output/'+str(date.today())+'_batch'+str(batch_number)) #generates the folder based on the dynamic date and batch number

for i, section in enumerate(config.sections()[1:]): #runs through each table in the config and generates the outputs
    table_config = dict(config.items(section))
    table = table_generate(i+1,paper_config,table_config,section,batch_number)
    table.make_all()